In [1]:
import os
import time
import random
import shutil
import subprocess
import numpy as np
import pandas as pd
import multiprocessing
from datetime import datetime
from itertools import islice
from collections import defaultdict, namedtuple
from tuplestate import init_from_solvitaire, to_dict, pprint_st
from benchmarking import *
from solver import solve

In [2]:
def get_state(ret):
    deck_json = convert_shootme_to_solvitaire_json(ret)
    return init_from_solvitaire(deck_json)


def prepare_ui_state(ret):
    parsed = parse_winnable(ret)
    game_state = to_dict(get_state(ret))
    
    def card(c):
        ''' UI uses 10 instead of T '''
        return c.replace('t', '10').replace('T', '10')
    
    return {
        'stock': [card(c).lower() for c in game_state['stock']],
        'tableau': [
            [card(c) for c in tab] 
            for tab in game_state['tableau']
        ],
        'waste': game_state['waste'],
        'foundation': game_state['foundation'],
        'moveSeq': parsed['moves'].strip().split(' ')
    }

## Read benchmark output

In [3]:
results = defaultdict(set)

all_solutions = os.listdir('./bench/shootme/')
seeds = list(map(lambda fname: int(fname[:-4]), all_solutions))

def solve_state(ret):
    lines = ret.splitlines()
    result = lines[15]
    if result.startswith('Minimal solution'):
        return "Solved-Min"
    elif result.startswith("Solved"):
        return "Solved"
    elif result.startswith('Impossible'):
        return "Impossible"
    elif result.startswith('Unknown'):
        return "Unknown"

for seed in sorted(seeds):
    with open(f"bench/shootme/{seed}.txt") as f:
        ret = f.read()
        result = solve_state(ret)
        results[result].add(seed)
    
seed_class = {}
for cls in results.keys():
    seeds_cls = results[cls]
    for seed in seeds_cls:
        seed_class[seed] = cls
        
for res, seeds in results.items():
    print(f"{res:12} {len(seeds):8,}")
total = sum(len(s) for s in results.values())
print(('-'*12) + '-' + ('-'*8))
print(f"{'Total':12} {total:8,}")

Solved-Min      7,876
Impossible      1,282
Unknown           522
Solved            320
---------------------
Total          10,000


## Generate suites

In [4]:
# seeds = np.fromiter(seed_class.keys(), dtype=np.int16)
# rand = np.random.RandomState(0)

# def get_suites(size, max_suites):
#     rand.shuffle(seeds)
#     start = 0
#     suites = 0
#     for s in range(max_suites):
#         end = start + size
#         if end > len(seeds):
#             return
#         yield list(map(int, seeds[start:end]))
#         start = end
#         suites += 1

# size = 10
# for i, suite in enumerate(get_suites(size=size, max_suites=1000)):
#     with open(f'./bench/suites/{size}/suite_{i:0>4}.txt', 'w') as f:
#         out = ' '.join(map(str,suite))
#         f.write(out)

## Run suites

In [5]:
def map_seeds_to_states(seed_seq):
    for seed in seed_seq:
        with open(f"bench/shootme/{seed}.txt") as f:
            ret = f.read()
            state = get_state(ret)
            yield seed, state

def get_seeds(fname):
    with open(fname) as f:
        return map(int, f.read().strip().split(' '))
    
    
def get_suite_states(fname):
    return list(map_seeds_to_states(get_seeds(fname)))


def get_suite_files(size):
    pref = f'./bench/suites/{size}/'
    ls = sorted(os.listdir(pref))
    if len(ls) == 0:
        raise Exception(f"no suite files in {pref}")
    return [os.path.join(pref, f) for f in ls]

In [6]:
Result = namedtuple('Result', 
    ['seed', 'time', 
     'solved', 'visited', 'msg',
     'seq', 'seqlen', 
     'datetime', 'shootme']
)

def run_solver(seedstate):
    seed, state = seedstate
#     print(f"starting seed {seed}")
    start = time.time()
    sol = solve(state, max_states=200_000, k=3)
    seq = None
    seqlen = -1
    if sol.solved:
        seq = " ".join(sol.moveseq)
        seqlen = len(seq)
    end = time.time()
    elapsed = end-start
    shootme = seed_class[seed]
    now = datetime.now().isoformat()
#     print(f"done solving {seed} after {elapsed:.1f}")
    return Result(
        seed=seed, time=elapsed,
        solved=sol.solved,
        visited=sol.visited,
        msg=sol.msg,
        seq=seq, seqlen=seqlen, 
        datetime=now, shootme=shootme
    )

In [7]:
proc = subprocess.Popen("git log --pretty=oneline | head -c 10", shell=True, stdout=subprocess.PIPE)
out, err = proc.communicate()
git = out.decode('ascii')
foldername = f"suite-10-rollout-k3-{git}"
folderpath = os.path.join("./bench/", foldername)

In [8]:
try:
    shutil.rmtree(folderpath)
except FileNotFoundError:
    pass
print(folderpath)

def timestr():
    now = datetime.now()
    return now.strftime("%Y%m%d-%H%M%S")

def save_results(results, suite_file):
    df = pd.DataFrame(results)
    fname = os.path.join(folderpath, f"{suite_file}-{timestr()}.csv")
    df.to_csv(fname)

./bench/suite-10-rollout-k3-014d25000e


In [ ]:
if not os.path.exists(folderpath):
    os.makedirs(folderpath)

with multiprocessing.Pool() as pool:
    for suite_file in get_suite_files(size=10):
#         print(f"starting {suite_file}")
        states = get_suite_states(suite_file)
        ret = pool.map(run_solver, states)
        _, fname = os.path.split(suite_file)
        fname, _ = os.path.splitext(fname)
        save_results(ret, fname)
        print(f'done {suite_file}')

done ./bench/suites/10/suite_0000.txt
done ./bench/suites/10/suite_0001.txt
done ./bench/suites/10/suite_0002.txt
done ./bench/suites/10/suite_0003.txt
done ./bench/suites/10/suite_0004.txt
done ./bench/suites/10/suite_0005.txt
done ./bench/suites/10/suite_0006.txt
done ./bench/suites/10/suite_0007.txt
done ./bench/suites/10/suite_0008.txt
done ./bench/suites/10/suite_0009.txt
done ./bench/suites/10/suite_0010.txt
done ./bench/suites/10/suite_0011.txt
done ./bench/suites/10/suite_0012.txt
done ./bench/suites/10/suite_0013.txt
done ./bench/suites/10/suite_0014.txt
done ./bench/suites/10/suite_0015.txt
done ./bench/suites/10/suite_0016.txt
done ./bench/suites/10/suite_0017.txt
done ./bench/suites/10/suite_0018.txt
done ./bench/suites/10/suite_0019.txt
done ./bench/suites/10/suite_0020.txt
done ./bench/suites/10/suite_0021.txt
done ./bench/suites/10/suite_0022.txt
done ./bench/suites/10/suite_0023.txt
done ./bench/suites/10/suite_0024.txt
done ./bench/suites/10/suite_0025.txt
done ./bench

In [ ]:
# suite_file = './bench/suites/10/suite_0027.txt'
# states = get_suite_states(suite_file)
# ret = list(map(run_solver, states))
# ret